## Animal Services Report
### Introduction
### Thesis
### Glossary

In [9]:
import sqlite3
import pandas as pd
# Create sqlite database
conn = sqlite3.connect('animal_services.db')
# Read CSV into a variable
animalservices_data = pd.read_csv('louisville_animal_services_data.csv')
# Use as SQL
animalservices_data.to_sql('animalservices_data', conn, if_exists='replace')


In [2]:
# This cell will be my playground for now
pd.read_sql_query("""
    SELECT AnimalType, PrimaryColor FROM animalservices_data
    LIMIT 10;
""", conn)

,AnimalType,PrimaryColor
0,CAT,WHITE
1,CAT,BLACK
2,BIRD,RED
3,OTHER,BROWN
4,DOG,WHITE
5,BIRD,BLACK
6,DOG,BROWN
7,CAT,BROWN TABBY
8,DOG,WHITE
9,CAT,GRAY


In [8]:
# How many different types of animals were brought in?
pd.read_sql_query("""
    SELECT DISTINCT AnimalType as 'Animal Types', 
    COUNT(*) as 'Count' 
    FROM animalservices_data
    GROUP BY AnimalType;
""", conn)

,Animal Types,Count
0,BIRD,1185
1,CAT,68741
2,DOG,77145
3,FERRET,91
4,LIVESTOCK,256
5,OTHER,1341
6,RABBIT,1139
7,REPTILE,263
8,RODENT,681


In [14]:
# Black cat query... don't break mah heart:
pd.read_sql_query("""
    SELECT DISTINCT PrimaryColor as 'Cat Color', 
    COUNT(*) as 'Count' 
    FROM animalservices_data
    GROUP BY PrimaryColor
    ORDER BY Count DESC
    LIMIT 10;
""", conn)

,Cat Color,Count
0,BLACK,42259
1,WHITE,17379
2,BROWN,16764
3,GRAY,11566
4,TAN,7722
5,BROWN TABBY,7015
6,GRAY TABBY,4301
7,CALICO,3980
8,TRICOLOR,3892
9,ORANGE TABBY,3839


In [21]:
# More heart breaking...
pd.read_sql_query("""
    SELECT 
    DISTINCT OutcomeType as 'Outcome', 
    COUNT(*) as 'Count' 
    FROM animalservices_data
    GROUP BY OutcomeType
    ORDER BY Count DESC
    ;
""", conn)

,Outcome,Count
0,EUTH,69662
1,ADOPTION,25724
2,TRANSFER,19223
3,RTO,14468
4,FOSTER,4654
5,RELEASED,3712
6,TNR,3310
7,DIED,2637
8,DISPOSAL,1933
9,TRANSPORT,1668


In [22]:
#OutcomeAsilomarStatus
pd.read_sql_query("""
    SELECT 
    DISTINCT OutcomeAsilomarStatus as 'Status at Outcome', 
    COUNT(*) as 'Count' 
    FROM animalservices_data
    GROUP BY OutcomeAsilomarStatus
    ORDER BY Count DESC
    ;
""", conn)

,Status at Outcome,Count
0,HEALTHY,147485
1,UNHEALTHY/UNTREATABLE,2765
2,TREATABLE/MANAGEABLE,390
3,None,202


In [33]:
# Cat breeds
pd.read_sql_query("""
    SELECT 
    DISTINCT PrimaryBreed as 'Breed', 
    COUNT(*) as 'Count' 
    FROM animalservices_data
    WHERE AnimalType = 'CAT'
    GROUP BY PrimaryBreed
    ORDER BY Count DESC
    LIMIT 10
    ;
""", conn)

,Breed,Count
0,DOMESTIC SHORTHAIR,55161
1,DOMESTIC MEDIUMHAIR,5986
2,DOMESTIC LONGHAIR,3295
3,AMERICAN SHORTHAIR,1607
4,SIAMESE,1281
5,MAINE COON,290
6,RUSSIAN BLUE,156
7,BENGAL,130
8,PERSIAN,129
9,SNOWSHOE,126


In [32]:
# Dog breeds
pd.read_sql_query("""
    SELECT 
    DISTINCT PrimaryBreed as 'Breed', 
    COUNT(*) as 'Count' 
    FROM animalservices_data
    WHERE AnimalType = 'DOG'
    GROUP BY PrimaryBreed
    ORDER BY Count DESC
    LIMIT 10
    ;
""", conn)

,Breed,Count
0,PIT BULL TERRIER,14911
1,LABRADOR RETRIEVER,9615
2,GERMAN SHEPHERD DOG,5495
3,BEAGLE,5347
4,BOXER,2897
5,CHIHUAHUA - SMOOTH COATED,2621
6,AMERICAN PIT BULL TERRIER,2312
7,ROTTWEILER,2264
8,CHOW CHOW,2203
9,BORDER COLLIE,1635


In [42]:
# IntakeReason per Cat
pd.read_sql_query("""
    SELECT 
    DISTINCT IntakeReason as 'Intake Reason', 
    COUNT(*) as 'Count' 
    FROM animalservices_data
    WHERE AnimalType = 'CAT'
    GROUP BY IntakeReason
    ORDER BY Count DESC
    LIMIT 10
    ;
""", conn)

,Intake Reason,Count
0,None,53141
1,STRAY,6193
2,TOO MANY,1678
3,MOVE,1100
4,COST,1029
5,LANDLORD,718
6,ALLERGIC,683
7,OWNER PROB,667
8,NO HOME,273
9,OTHER PET,244


### Analysis
### Conclusion